# Perform Window CRQA on IAAFT time-series

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_window_surrogates(original_window, surrogates_list, n_to_plot=2, column=None):
    """
    This is a modified version found in iaaft example.py created to plot original and surrogates on a window-by-window basis.

    Plot original time series, surrogate time series, and power spectra
    for a single window.

    Parameters
    ----------
    original_window : pd.DataFrame
        Original windowed time series (one or multiple columns)
    surrogates_list : list of pd.DataFrame
        List of surrogate time series (same shape as original_window)
    n_to_plot : int
        Number of surrogate series to plot
    """
    # For simplicity, use the first column if multiple columns
    if column is not None: 
        col = column
        x = original_window[col].values
        n = len(x)
    else:
        col = original_window.columns[0]
        x = original_window[col].values
        n = len(x)

    # Compute original power spectrum
    p = np.square(np.abs(np.fft.fft(x)))
    freq = np.fft.fftfreq(n)

    # Convert surrogates_list to array
    ns = len(surrogates_list)
    xs = np.array([s[col].values for s in surrogates_list])
    ps = np.zeros((ns, n))
    for i in range(ns):
        ps[i] = np.square(np.abs(np.fft.fft(xs[i])))

    # Select random surrogates to plot
    idx = np.random.randint(ns, size=n_to_plot)
    clrs = ["coral", "teal", "goldenrod"]

    AXLABFS, TIKLABFS = 14, 11
    fig = plt.figure(figsize=[12., 6.])

    fig.suptitle(f"{col}", fontsize=16)

    ax1 = fig.add_axes([0.10, 0.55, 0.375, 0.35])
    ax2 = fig.add_axes([0.10, 0.10, 0.375, 0.35])
    ax3 = fig.add_axes([0.55, 0.10, 0.40, 0.80])

    # Original time series
    ax1.plot(x, "steelblue", label="original")

    # Surrogate time series
    for j, i in enumerate(idx):
        ax2.plot(xs[i], c=clrs[j], label=f"surrogate #{i}", alpha=0.45)

    # Power spectra
    k = np.argsort(freq)
    k = k[int(len(k)/2):]  # positive frequencies
    ax3.plot(freq[k], p[k], "o-", mec="steelblue", mfc="none",
             label="original", alpha=0.45)
    for j, i in enumerate(idx):
        ax3.plot(freq[k], ps[i, k], "x-", mec=clrs[j], mfc="none",
                 label=f"surrogate #{i}", alpha=0.45)
    ax3.set_yscale("log")

    # Beautify
    for ax in fig.axes:
        leg = ax.legend(loc="upper right")
        for txt in leg.get_texts():
            txt.set_size(TIKLABFS)
        ax.tick_params(labelsize=TIKLABFS)
    for ax in [ax1, ax2]:
        ax.set_ylabel("Signal", fontsize=AXLABFS)
        ax.set_xlabel("Time", fontsize=AXLABFS)
    ax3.set_xlabel("Frequency", fontsize=AXLABFS)
    ax3.set_ylabel("Power", fontsize=AXLABFS)

    plt.show()


In [ ]:
# Import necessary libraries
import pandas as pd
from utils_rqa.crossRQA import crossRQA
import os
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

# Features (time series variables) to include in the CRQA
pos_cols = ['head_ed_pos',
            'headRel_ed_pos',
            'body_ed_pos'
            ]

acc_cols = ['head_ed_acc', 'body_ed_acc',
            'headRel_ed_acc']

vel_cols = ['head_ed_vel', 'headRel_ed_vel', 'body_ed_vel', 
            'whole_all_vel', 'head_all_vel', 'body_all_vel',
            'head_movement_norm', 'body_movement_norm', 'full_body_movement_norm']

analysed_cols = ['headRel_ed_vel', 'body_ed_vel'] # Signals analysed and reported on.

In [ ]:
# Load partner role metadata
roles = pd.read_csv('partner_role_metadata.csv')

# Set directories
directory = "/..../Processed_Timeseries" # Set directory to processed timeseries.
output_dir = "crqa_output" # Output Directory
os.makedirs(output_dir, exist_ok=True)

# Create list of files. 
files = [f for f in os.listdir(directory) if f.split('_')[1] == 'P1']

# Choose which set of columns to use:
cols = analysed_cols 

In [ ]:
"""
Parameters used for the paper include:
    - Delay = 5
    - Embedding = 4
    - Radii = 0.25
    - min line = 2
"""
# Parameters for delay embedding + recurrence analysis
delays = [5]
embedding = [4]
radii = [0.25]

# Degine the number of Surrogate IAAFT time series to generate and perform CRQA on.
num_surrogates = 10

display_transformations = False # Display and compare transformed time series to original time series.

# Loop through parameters and files to perform windowed cross-recurrence quantification analysis (CRQA)
for d in tqdm(delays, desc='Delays Processed'):
    for m in tqdm(embedding, desc='Embedding Processed', leave=False):
        for r in tqdm(radii, desc="Radii Processed", leave=False):
            results = [] # Store results for all couples/trials/windows

            # RQA parameters
            crqa_params = {
                'norm': 2, # 2 = zscore, 1 = unit
                'eDim': m,
                'tLag': d,
                'rescaleNorm': 1, # 1 = mean rescale, 2 = max
                'radius': r,
                'tw': 0,
                'minl': 2,
                'doPlots': False,
                #'plotMode': 'rp-timeseries', # 'rp' or 'rp-timeseries'
                'pointSize': 2, 
                'saveFig': False,
                'showMetrics': False,
                'doStatsFile': False
            }
            # Loop through files
            for file in files: 
                couple = file.split('_')[0] # Extract couple ID from filename
                trial = file.split('_')[4]  # Extract trial number from filename

                # Load participant 1 data
                p1_data_path = os.path.join(directory, file)
                p1_data = pd.read_csv(p1_data_path)

                 # Load corresponding participant 2 data
                p2_data_path = os.path.join(directory, file.replace('P1', 'P2'))

                # Check if the P2 file exists before attempting to read it
                if os.path.exists(p2_data_path):
                    p2_data = pd.read_csv(p2_data_path)
                else:
                    print(f"File does not exist: {file.replace('P1', 'P2')}")

                # Debug print of paired files
                print(f"selected file: {file}")
                print(f"partner file: {file.replace('P1', 'P2')}")

                # Extract window indices for both participants 
                p1_win_index = p1_data['Window_Index'].values
                p2_win_index = p2_data['Window_Index'].values

                # Find common window indices between both participants
                matching_windows = np.intersect1d(p1_win_index, p2_win_index)

                # Subset data to only include matching windows
                p1_col = p1_data[p1_data['Window_Index'].isin(matching_windows)]
                p2_col = p2_data[p2_data['Window_Index'].isin(matching_windows)]

                # Skip if window counts do not match
                if len(p1_col) != len(p2_col):
                    print(f"Unequal length. Couple = {couple}, trial = {trial}")
                    continue
    
                # Loop through each matching window and perform CRQA
                i = 0
                for idx in matching_windows:
                    surrogate_outputs = []
                    p1_window_surr_list = []
                    i += 1
                    p1_window = p1_col[p1_col['Window_Index'] == idx][cols].iloc[1:]
                    p2_window = p2_col[p2_col['Window_Index'] == idx][cols].iloc[1:]

                    for n in range(num_surrogates):
                        p1_window_surr = p1_window.copy()
                        p2_window_surr = p2_window.copy()

                        for col in cols:
                            p1_window_surr[col] = surrogates(x=p1_window[col].values, ns=1, verbose=False)[0]
                            p2_window_surr[col] = surrogates(x=p2_window[col].values, ns=1, verbose=False)[0]
                        
                        # print(p1_window_surr)
                        if display_transformations:
                            p1_window_surr_list.append(p1_window_surr)
                        

                        file_info = {
                            'couple': couple,
                            'trial': trial,
                            'window_index': idx,
                            'surrogate': n+1

                            }
                        
                        try: 
                            output = crossRQA(p1_window_surr, p2_window_surr, crqa_params, file_info=file_info)
                        except Exception as ex:
                            print('error')
                            file_info['err_code'] = 1
                            output = file_info
                            template = "An exception of type {0} occurred. Arguments:\n{1!r}"
                            message = template.format(type(ex).__name__, ex.args)
                            print(f'{message}: For file: {file}')
                        
                        surrogate_outputs.append(output)
                    if display_transformations:
                        plot_window_surrogates(p1_window, p1_window_surr_list, n_to_plot=2, column='headRel_ed_vel')

                    surrogate_df = pd.DataFrame(surrogate_outputs)
                    numeric_cols = surrogate_df.select_dtypes(include='number').columns

                    averaged_metrics = surrogate_df[numeric_cols].mean().to_dict()
                    std_metrics = surrogate_df[numeric_cols].std().add_suffix('_std').to_dict()

                    averaged_metrics.update(std_metrics)

                    averaged_metrics.update({
                        'couple': couple,
                        'trial': trial,
                        'window_index': idx,
                        'n_surrogates': num_surrogates
                    })

                    results.append(averaged_metrics)

            # Save results for current parameter combination
            csv_df = pd.DataFrame(results)
            csv_df.to_csv(f"{output_dir}/PseudoCrossRqa_win_delay{d}_dim{m}_rad{r}_minl{crqa_params['minl']}_2.csv", index=False)      

            print('Windowed CRQA analysis and plotting completed successfully!')